# Implementing A GPT-model From Scratch

- Coding a GPT-like LLM that can be trained to generate uman-like text.
- Normalizing layer activations to stabilize NN training.
- Adding `shortcut connections` to train deep NNs effectively.
- Implementing transformer blocks to create GPT models of various sizes.
- Computing the number of parameters and storage requiremnts of GPT models.

In [ ]:
%load_ext watermark
%watermark -v -p numpy,pandas,polars,torch,lightning --conda

In [ ]:
# Built-in library
from pathlib import Path
import re
import json
from typing import Any, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
import numpy.typing as npt
from pprint import pprint
import pandas as pd
import polars as pl
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "info": "#76FF7B",
        "warning": "#FBDDFE",
        "error": "#FF0000",
    }
)
console = Console(theme=custom_theme)

# Visualization
import matplotlib.pyplot as plt


# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

warnings.filterwarnings("ignore")


# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from torch import nn, Tensor
import torch.nn.functional as F

In [ ]:
GPT_CONFIG_124M: dict[str, Any] = {
    "vocab_size": 50_257,
    "context_length": 1_024,
    "emb_dim": 768,
    "n_heads": 12,  # Number of attention heads
    "n_layers": 12,
    "drop_rate": 0.1,  # Dropout rate
    "qkv_bias": False,
}

### A Placeholder GPT Backbone

In [ ]:
class DummyGPTModel(nn.Module):
    def __init__(self, cfg: dict[str, Any]) -> None:
        super().__init__()

        self.tok_emb = nn.Embedding(cfg.get("vocab_size"), cfg.get("emb_dim"))
        self.pos_emb = nn.Embedding(cfg.get("context_length"), cfg.get("emb_dim"))
        self.drop_emb = nn.Dropout(cfg.get("drop_rate"))
        self.transformer_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg.get("n_layers"))]
        )

        self.final_norm = DummyLayerNorm(cfg.get("emb_dim"))
        self.out_head = nn.Linear(cfg.get("emb_dim"), cfg.get("vocab_size"), bias=False)

    def forward(self, x: Tensor) -> Tensor:
        batch_size, seq_len = x.shape
        tok_embeds: Tensor = self.tok_emb(x)
        pos_embeds: Tensor = self.pos_emb(torch.arange(seq_len, device=x.device))
        x = tok_embeds + pos_embeds  # Add token and positional embeddings to get input embeddings
        x = self.drop_emb(x)
        x = self.transformer_blocks(x)
        x = self.final_norm(x)
        logits: Tensor = self.out_head(x)
        return logits


class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg: dict[str, Any]) -> None:
        super().__init__()

    def forward(self, x: Tensor) -> Tensor:
        return x


class DummyLayerNorm(nn.Module):
    def __init__(self, cfg: dict[str, Any]) -> None:
        super().__init__()

    def forward(self, x: Tensor) -> Tensor:
        return x

### Overview

<img src="../08-Makemore/images/ch04_mental_.gpt_picpng.png" alt="gpt" width="600">

[image source](https://livebook.manning.com/book/build-a-large-language-model-from-scratch/chapter-4/v-7/36)

In [ ]:
# Tokenize a batch consisting of sample texts.
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
text1: str = "Every effort moves you"
text2: str = "Everyday holds a"
batch: list[int] = []

batch.append(torch.tensor(tokenizer.encode(text1)))
batch.append(torch.tensor(tokenizer.encode(text2)))
batch_tensor: Tensor = torch.stack(batch, dim=0)

print(f"Shape of batch_tensor: {tuple(batch_tensor.shape)}\n")
print(f"Batch_tensor: \n{batch_tensor}\n")

In [ ]:
# Instantiate the DummyGPTModel
seed: int = 5
torch.manual_seed(seed)

model = DummyGPTModel(GPT_CONFIG_124M)
logits: Tensor = model(batch_tensor)

print(f"Shape of logits: {tuple(logits.shape)}\n")
print(f"logits: \n{logits}\n")

### Comment

- The output (2, 4, 50_257) has `2` rows corresponding to the number of input data.
- It has `4` rows which is the context length of the input and `50_257` is the number of unique words (vocab_size) in the input.

### Normalizing Activations With Layer Normalization